
<div style="text-align: center; margin-top: 20px;">
  <a href="http://quantum.analyticsdojo.com">
    <img src="https://github.com/AnalyticsDojo/applied-quantum/blob/main/site/both-logo-red.png?raw=1"
         alt="Analytics Dojo"
         style="width: 250px; max-width: 100%; height: auto;">
  </a>
  <h1 style="margin: 10px 0;">Knapsack Problem (Sample Starter Project)</h1>
  <h3>
    <a href="http://quantum.analyticsdojo.com" style="text-decoration: none; color: #b30000;">
      quantum.analyticsdojo.com
    </a>
  </h3>
</div>



# 1. Knapsack Problem (Sample Starter Project)

This is a sample structure for your final project. The goal is to go through the detailed explanation of the problem you are working with. This is a very short example. Please go into much more detail.

**What’s the problem?**  
We have a backpack (the “knapsack”) that can only hold so much weight. We also have a bunch of items.  
Each item has a **value** (how useful/profitable it is) and a **weight** (how heavy it is).  
We want to **choose a set of items** to put in the backpack so that the **total value is as big as possible**,
**without** the total weight being **too heavy** for the backpack.

**Real-world motivation**  
This shows up anytime you must **pick the best combination** under a **limited resource**:
- A delivery truck with a weight limit choosing which packages to carry.
- A budget-constrained investor deciding which projects to fund.
- A satellite choosing which photos to transmit given bandwidth limits.


# 2. Fitness Function and Dependent Variable

**Goal (objective):** Maximize the **total value** of chosen items while staying within a weight capacity.

**Fitness function (to maximize):**

<div align="center">

$$
f(\mathbf{x}) = \sum_i v_i x_i \quad \text{subject to} \quad \sum_i w_i x_i \le C
$$

</div>

where:

- \(x_i \in \{0,1\}\) indicates whether item \(i\) is chosen,  
- \(v_i\) is the value of item \(i\),  
- \(w_i\) is the weight of item \(i\),  
- \(C\) is the knapsack capacity.  

**Dependent variable:**  
The **total value** (or total profit) **depends** on our binary decisions \(x_i\).  
If we choose different items, the total value changes. So the **dependent variable** is the **objective value**.

# 3. Decision Variables

**Decision variables:**  
For each item \(i\), we define a binary variable \(x_i\):

- \(x_i = 1\): we include item \(i\) in the knapsack  
- \(x_i = 0\): we do **not** include item \(i\)

<div align="center">

$$
\mathbf{x} = (x_1, x_2, \dots, x_n)
$$

</div>

This vector \(\mathbf{x}\) encodes our chosen set of items.


# 4. (Optional) Research Context

- **Lucas, A. (2014).** *Ising formulations of many NP problems*. Frontiers in Physics, 2:5.  
  Shows how classical combinatorial problems (including knapsack) can be rewritten as Ising/QUBO problems suitable for quantum/annealing approaches.

- **Farhi, E., Goldstone, J., & Gutmann, S. (2014).** *A Quantum Approximate Optimization Algorithm (QAOA)*, arXiv:1411.4028.  
  Introduces QAOA, a general framework applied to a wide range of combinatorial optimization problems (including knapsack via QUBO encodings).


In [ ]:

# 5. Simple Python Demonstration of the Fitness Function

from itertools import combinations

# Example data (small so we can brute-force):
values  = [9, 6, 7, 3, 5]
weights = [4, 2, 3, 1, 2]
C = 7  # capacity

def fitness(x, values, weights, C):
    # Compute total value if feasible, else return -inf to indicate infeasible.
    total_w = sum(w*x_i for w, x_i in zip(weights, x))
    total_v = sum(v*x_i for v, x_i in zip(values, x))
    if total_w <= C:
        return total_v
    else:
        return float('-inf')

# Brute-force search over all subsets to demonstrate the concept:
best_x = None
best_val = float('-inf')

n = len(values)
for r in range(n+1):
    for idxs in combinations(range(n), r):
        x = [0]*n
        for i in idxs: x[i] = 1
        val = fitness(x, values, weights, C)
        if val > best_val:
            best_val = val
            best_x = x

print("Best subset (x):", best_x)
print("Best value:", best_val)
print("Total weight:", sum(w*x_i for w, x_i in zip(weights, best_x)))


Best subset (x): [1, 1, 0, 1, 0]
Best value: 18
Total weight: 7



# 6. BQP Version of the Optimization (QUBO Form)

We want an **unconstrained** quadratic problem over **binary** variables.

Start from the constrained maximization:
\[
\max_{\mathbf{x}\in\{0,1\}^n} \sum_i v_i x_i \quad \text{s.t.} \quad \sum_i w_i x_i \le C.
\]

Convert to an unconstrained **minimization** by negating the objective and adding a penalty for violating capacity.  
A robust way to keep everything quadratic is to introduce **binary slack bits** \( \mathbf{y} \) so that:
\[
\sum_i w_i x_i + \sum_{k=0}^{m-1} 2^k y_k = C,
\]
where \(m\) is chosen so that \(2^m-1 \ge C\).

Then minimize the QUBO:
\[
\min_{\mathbf{x},\mathbf{y}\in\{0,1\}} \underbrace{-\sum_i v_i x_i}_{\text{(negated value)}} \;+\;
P\Big(\sum_i w_i x_i + \sum_k 2^k y_k - C\Big)^2,
\]
with a penalty \(P\) chosen large enough so the constraint is respected at optimum (e.g., \(P > \max_i v_i\)).

Expanding the square yields a **Quadratic Unconstrained Binary Optimization (QUBO)** \( \mathbf{z}^\top Q \mathbf{z} + c \)
over the concatenated binary vector \( \mathbf{z} = [\mathbf{x}; \mathbf{y}] \).


In [ ]:
from userLibrary_helper import OptimizationProblem, PenaltyType, FitnessEvalParameter
from typing import List

def Evaluator(x):
    dimensionality = len(x)
    def objective(x: List[float], dimensionality: int) -> float:

        Values = [47, 107, 53, 139, 179, 106, 92, 93, 35, 48, 40, 175,
            111, 60, 158, 78, 179, 20, 3, 188, 82, 140, 7, 194, 35, 181, 157, 36, 35, 193, 63,
            200, 160, 100, 101, 94, 185, 144, 72, 112, 72, 71, 24, 49, 147, 31, 175, 156, 67,
            155, 128, 114, 122, 156, 94, 193, 120, 141, 177, 134, 195, 11, 49, 65, 54, 46, 185,
            180, 123, 174, 117, 147, 150, 23, 75, 192, 35, 80, 84, 78, 18, 98, 119, 141, 89, 90,
            98, 99, 119, 61, 200, 24, 157, 196, 199, 14, 48, 193, 124, 168, 112, 164, 132, 126,
            117, 193, 93, 1, 70, 112, 48, 71, 193, 181, 100, 150, 88, 81, 24, 30, 49, 178, 28,
            142, 119, 162, 137, 38, 30, 67, 150, 155, 81, 21, 31, 63, 58, 141, 84, 36, 125, 21,
            187, 68, 184, 49, 162, 190, 12, 116, 68, 180, 76, 75, 41, 21, 97, 154, 112, 93, 176,
            94, 157, 76, 200, 59, 191, 141, 55, 37, 162, 172, 87, 130, 12, 81, 86, 170, 66, 66,
            109, 55, 186, 96, 160, 116, 198, 38, 8, 147, 88, 68, 85, 193, 92, 27, 47, 135, 56,
            61, 18, 52, 71, 2, 122, 44, 53, 100, 133, 13, 59, 146, 133, 153, 32, 190, 157, 183,
            110, 38, 134, 181, 26, 62, 197, 184, 48, 49, 57, 195, 81, 161, 63, 10, 146, 29, 13,
            186, 87, 9, 197, 111, 29, 166, 130, 21, 41, 144, 132, 167, 148, 191, 174, 42, 83, 176,
            77, 20, 128, 74, 83, 48, 119, 45, 148, 87, 183, 127, 129, 21, 125, 104, 76, 122, 120,
            102, 79, 54, 91, 64, 31, 191, 156, 195, 74, 52, 147, 16, 90, 34, 77, 54, 109, 146, 154,
            102, 114, 21, 98, 93, 186, 112, 31, 51, 127, 57, 53, 118, 102, 165, 82, 138, 141, 127,
            185, 186, 4, 190, 175, 168, 118, 80, 181, 187, 46, 164, 8, 141, 171, 149, 116, 162, 69,
            156, 63, 63, 124, 51, 28, 113, 106, 29, 76, 147, 55, 21, 94, 101, 197, 158, 19, 158, 38,
            37, 174, 178, 105, 140, 23, 33, 67, 167, 51, 174, 197, 191, 93, 160, 107, 148, 167, 37,
            25, 52, 67, 97, 97, 1, 144, 51, 149, 72, 12, 19, 28, 61, 56, 172, 157, 115, 39, 43, 31,
            13, 114, 199, 189, 129, 115, 65, 181, 188, 138, 29, 52, 82, 163, 77, 105, 147, 110, 46,
            198, 109, 41, 187, 139, 151, 73, 94, 8, 22, 57, 109, 29, 103, 30, 116, 16, 190, 184, 155,
            96, 92, 126, 137, 183, 127, 4, 11, 169, 161, 89, 144, 138, 51, 80, 65, 156, 150, 132, 122,
            61, 23, 198, 3, 168, 120, 184, 159, 10, 52, 22, 163, 28, 189, 192, 130, 182, 96, 34, 56,
            174, 162, 175, 199, 146, 82, 35, 152, 115, 20, 74, 10, 94, 189, 94, 16, 68, 156, 175, 185,
            108, 17, 94, 21, 29, 34, 92, 34]

        Weights = [29, 76, 30, 86, 132, 83, 98, 175, 98, 141, 179, 74,
            24, 96, 87, 146, 43, 199, 177, 107, 60, 182, 41, 171, 31, 151, 181, 92, 197, 152,
            59, 43, 41, 109, 197, 41, 104, 48, 45, 22, 39, 89, 181, 189, 77, 19, 6, 159, 42,
            112, 40, 90, 161, 7, 40, 172, 128, 84, 90, 156, 35, 6, 76, 115, 127, 33, 86, 67,
            42, 19, 67, 191, 68, 8, 39, 75, 113, 180, 173, 132, 44, 197, 162, 159, 4, 88, 124,
            29, 159, 71, 151, 143, 167, 175, 76, 27, 121, 160, 181, 7, 37, 154, 73, 190, 112,
            95, 82, 40, 200, 92, 65, 129, 188, 185, 176, 13, 35, 29, 15, 23, 65, 78, 172, 6,
            24, 125, 193, 106, 34, 168, 93, 4, 15, 96, 9, 134, 42, 130, 17, 10, 49, 111, 44,
            21, 23, 34, 77, 124, 129, 56, 124, 129, 168, 185, 86, 43, 82, 104, 105, 135, 177,
            67, 86, 197, 140, 19, 8, 177, 32, 193, 19, 129, 50, 132, 190, 78, 27, 193, 59, 58,
            151, 72, 185, 8, 152, 11, 180, 179, 144, 197, 4, 183, 172, 149, 120, 40, 59, 1,
            50, 109, 78, 142, 68, 38, 61, 154, 102, 190, 173, 73, 31, 88, 23, 145, 80, 130,
            54, 11, 184, 180, 121, 17, 31, 77, 134, 190, 143, 51, 69, 117, 80, 123, 108, 189,
            82, 189, 174, 46, 71, 192, 127, 35, 138, 143, 172, 147, 118, 177, 47, 40, 144, 87,
            29, 189, 128, 142, 50, 191, 184, 175, 192, 167, 102, 71, 118, 168, 111, 179, 47,
            196, 123, 161, 132, 109, 17, 104, 13, 105, 177, 31, 154, 136, 114, 46, 14, 120,
            137, 55, 81, 24, 91, 18, 111, 85, 118, 107, 128, 162, 66, 90, 66, 143, 116, 35,
            191, 160, 187, 59, 62, 54, 52, 143, 30, 156, 161, 21, 133, 16, 198, 161, 155, 174,
            142, 29, 26, 103, 30, 22, 112, 162, 119, 171, 112, 118, 194, 45, 36, 103, 134,
            104, 184, 183, 169, 171, 90, 101, 103, 11, 135, 60, 21, 184, 139, 123, 129, 152,
            141, 193, 159, 143, 197, 43, 143, 117, 95, 43, 17, 31, 26, 135, 129, 36, 188, 184,
            22, 175, 67, 64, 190, 34, 132, 152, 50, 159, 147, 121, 96, 120, 8, 94, 52, 161,
            96, 198, 107, 59, 13, 69, 173, 182, 56, 178, 155, 159, 32, 50, 86, 80, 3, 6, 91,
            120, 93, 123, 186, 40, 29, 35, 136, 174, 56, 46, 62, 1, 140, 81, 174, 67, 43, 166,
            168, 45, 131, 128, 166, 78, 29, 136, 87, 136, 148, 20, 113, 79, 71, 119, 100, 8,
            55, 59, 75, 33, 139, 190, 109, 60, 122, 74, 119, 174, 140, 199, 68, 92, 137, 43,
            61, 173, 125, 161, 93, 124, 45, 36, 76, 171, 26, 199, 6, 59, 95, 24, 69, 18, 96,
            26, 94, 47, 85, 191, 17, 194, 20, 164, 11, 112, 22, 116, 143, 141]

        knapsackCapacity = 25233

        #------------------------ Define Objective functions ------------------------#
        objectiveFunctionValue = 0.0
        for i in range(0, dimensionality):
            objectiveFunctionValue += (Values[i] * x[i])
        objectiveFunctionValue = -objectiveFunctionValue

        #-------------------- Define multiple constraint functions--------------------#
        TotalWt=0
        for i in range(0, dimensionality):
            TotalWt += Weights[i] * x[i]

        # Constraint 1
        violation = 0.0
        ConstraintFunc = knapsackCapacity-TotalWt
        if ConstraintFunc < 0.0:
            violation += abs(ConstraintFunc)

        #------------------ Penalty coefficients for the constraints -------------------#

        penaltyCoefficients = [100.0]                                   # Coefficients equivalent to number of constraint
        Total_ConstraintViolation = penaltyCoefficients[0]*violation


        fitness = objectiveFunctionValue + Total_ConstraintViolation

        return fitness

    # Define constraint function

    # Create optimization problem
    constraints = []

    # Define penalty parameters
    penalty_type = "No_Penalty"             # Options: No_Penalty, Death_Penalty, Static_Penalty, Debs_Penalty
    # penalty_coefficients = [100.0, 100.0]       # Make sure number of entries are equivalent to number of constraints

    if penalty_type == "No_Penalty":
        ########### Using No-Penalty ###########
        problem = OptimizationProblem(objective, constraints, PenaltyType.No_Penalty)

    elif penalty_type == "Death_Penalty":
        ########### Using Death Penalty ############
        problem = OptimizationProblem(objective, constraints, PenaltyType.Death_Penalty)

    elif penalty_type == "Static_Penalty":
        ########### Using Static Penalty with a fixed penalty value ############
        problem = OptimizationProblem(objective, constraints, PenaltyType.Static_Penalty, penalty_coefficients)

    elif penalty_type == "Debs_Penalty":
        ########### Using Deb's approach ###########
        problem = OptimizationProblem(objective, constraints, PenaltyType.Debs_Penalty)

    return problem.evaluate_objective(x, dimensionality)



### Results

Describe the results that you find (include some pictures) from the results.
